In [1]:
import numpy as np

In [2]:
import os
path="C:\\Users\\Anil\\Downloads\\20_newsgroups-20190710T162656Z-001\\20_newsgroups"
#all the folders in directory
folder=os.listdir(path)
folder

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
x=[]#files in folders
y=[]#folders
for fname in folder:
    f_path=path+"\\"+fname
    files=os.listdir(f_path)#files in current folder
    for cur_file in files:
        x.append(cur_file)
        y.append(fname)

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0)

In [5]:
from nltk.corpus import stopwords
stops=set(stopwords.words('english'))
len(stops)

179

In [6]:
from string import punctuation
def removepunc(word):
    for p in punctuation:
        word.replace(p,'')
    return word

In [7]:
#buld vocabulary
vocabulary={}
for i in range(len(x_train)):
    cur_path=path+"\\"+y_train[i]+"\\"+x_train[i]
    fileopen=open(cur_path,'r',errors='ignore')#open file
    record=fileopen.read()
    words=record.split()
    for cur_word in words:
        cur_word=removepunc(cur_word)
        if(len(cur_word)>2):
            if(cur_word.lower() in stops):
                continue
            else:
                if(cur_word.lower() in vocabulary):
                    vocabulary[cur_word.lower()]+=1
                else:
                    vocabulary[cur_word.lower()]=1
    fileopen.close()

In [8]:
len(vocabulary)

355496

In [9]:
#finding frequencies
freq=sorted(vocabulary.values())
Y=set(freq)
Y=list(Y)
wordcount=[]
for i in Y:
    wordcount.append(sum(value==i for value in freq))

In [10]:
#select top 3000 features
selected_feature=[]
import operator
sorted_vocab = sorted(vocabulary.items(), key=operator.itemgetter(1)) #sorting the vocabulary wrt the frequency
vocab_len=len(sorted_vocab)
for i in range(vocab_len-1,vocab_len-3000-1,-1):
    selected_feature.append(sorted_vocab[i][0])
len(selected_feature)

3000

In [11]:
#Converting the given data as required
data=np.ndarray(shape=(len(x_train),len(selected_feature)))
for i in range(len(x_train)):
    count=np.zeros(len(selected_feature))
    cur_path=path+"\\"+y_train[i]+"\\"+x_train[i]
    fileopen=open(cur_path,'r',errors='ignore')
    record=fileopen.read()
    words=record.split()
    for cur_word in words:
        cur_word=removepunc(cur_word)
        col=0
        for j in range(len(selected_feature)):
            if(cur_word.lower()==selected_feature[j]):
                count[j]+=1
                col=j
                break
        
        data[i][col]=count[col]

In [12]:
import pandas as pd
df=pd.DataFrame(data)
df.columns=selected_feature

In [13]:
df["y"]=y_train
df.head()

,from:,subject:,date:,newsgroups:,message-id:,lines:,path:,apr,organization:,gmt,...,learning,was.,accounts,wood,officer,artificial,(of,cities,automatically,y
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sci.space
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,talk.politics.mideast
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,rec.sport.baseball
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sci.electronics
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,misc.forsale


In [15]:
#convert testing data to required format
data=np.ndarray(shape=(len(x_test),len(selected_feature)))
for i in range(len(x_test)):
    count=np.zeros(len(selected_feature))
    cur_path=path+"\\"+y_test[i]+"\\"+x_test[i]
    fileopen=open(cur_path,'r',errors='ignore')
    record=fileopen.read()
    words=record.split()
    for cur_word in words:
        cur_word=removepunc(cur_word)
        col=0
        for j in range(len(selected_feature)):
            if(cur_word.lower()==selected_feature[j]):
                count[j]+=1
                col=j
                break
        
        data[i][col]=count[col]
        
df_test=pd.DataFrame(data)
df_test.columns=selected_feature
df_test["y"]=y_test
df_test.head()

,from:,subject:,date:,newsgroups:,message-id:,lines:,path:,apr,organization:,gmt,...,learning,was.,accounts,wood,officer,artificial,(of,cities,automatically,y
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,soc.religion.christian
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sci.med
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,talk.politics.guns
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,talk.politics.mideast
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sci.electronics


In [24]:
def fit(x_train,y_train):
    result={}
    result["total_data"]=len(y_train)
    classes=set(y_train)
    for cur_class in classes:
        x_train_rows=x_train[(y_train==cur_class)]
        y_train_rows=y_train[(y_train==cur_class)]
        result[cur_class]={}
        result[cur_class]["total_count"]=len(y_train_rows)
        num_features=x_train.shape[1]
        for i in range (1,num_features+1):
            result[cur_class][i]=x_train_rows[:,i-1].sum()
            
    return result

In [17]:
def probability(x,cur_class,result):
    output=np.log(result[cur_class]["total_count"])-np.log(result["total_data"])
    num_features=len(result[cur_class].keys())-1
    for i in range(1,num_features+1):
        if(x[i-1]==0):
            continue;
        count_curclass_xi=result[cur_class][i]+1
        total_count=result[cur_class]["total_count"]+len(result[cur_class])
        cur_prob=np.log(count_curclass_xi)-np.log(total_count)
        output=output+cur_prob
    return output

In [18]:
def predictsinglepoint(x,result):
    classes=result.keys()
    best_p=-1000
    best_class=-1;
    for cur_class in classes:
        if(cur_class=="total_data"):
            continue;
        val=probability(x,cur_class,result)
        if(val>best_p):
            best_p=val
            best_class=cur_class
    return best_class

In [19]:
def predict(x_test,result):
    y_pred=[]
    for x in x_test:
        y_pred.append(predictsinglepoint(x,result))
    return y_pred

In [20]:
k=df.values

In [21]:
x_train_features=k[:,:3000]

In [25]:
result=fit(x_train_features,np.array(y_train))

In [26]:
j=df_test.values

In [27]:
x_test_features=j[:,:3000]

In [28]:
y_predicted=predict(x_test_features,result)

In [29]:
from sklearn.metrics import classification_report, confusion_matrix

In [30]:
print(classification_report(y_test,y_predicted))
print(confusion_matrix(y_test,y_predicted))

                          precision    recall  f1-score   support

                      -1       0.00      0.00      0.00         0
             alt.atheism       0.74      0.76      0.75       233
           comp.graphics       0.78      0.63      0.70       253
 comp.os.ms-windows.misc       0.98      0.53      0.69       249
comp.sys.ibm.pc.hardware       0.82      0.70      0.76       240
   comp.sys.mac.hardware       0.98      0.53      0.69       236
          comp.windows.x       0.77      0.83      0.80       246
            misc.forsale       0.94      0.40      0.57       252
               rec.autos       0.94      0.47      0.62       267
         rec.motorcycles       1.00      0.39      0.56       255
      rec.sport.baseball       0.99      0.64      0.78       251
        rec.sport.hockey       0.98      0.86      0.92       239
               sci.crypt       0.37      0.91      0.53       229
         sci.electronics       0.99      0.26      0.41       285
         

C:\Users\Anil\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [37]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(x_train_features,np.array(y_train))
y_pred2=clf.predict(x_test_features)

In [38]:
print(classification_report(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))

                          precision    recall  f1-score   support

             alt.atheism       0.70      0.78      0.74       233
           comp.graphics       0.78      0.74      0.76       253
 comp.os.ms-windows.misc       0.87      0.80      0.83       249
comp.sys.ibm.pc.hardware       0.84      0.88      0.86       240
   comp.sys.mac.hardware       0.84      0.90      0.87       236
          comp.windows.x       0.88      0.84      0.86       246
            misc.forsale       0.69      0.91      0.79       252
               rec.autos       0.82      0.90      0.86       267
         rec.motorcycles       0.85      0.96      0.90       255
      rec.sport.baseball       0.97      0.96      0.96       251
        rec.sport.hockey       0.99      0.96      0.97       239
               sci.crypt       0.96      0.87      0.92       229
         sci.electronics       0.81      0.88      0.84       285
                 sci.med       0.93      0.86      0.89       231
         